In [1]:
# https://stackoverflow.com/a/67029719/7782
from IPython import get_ipython
from IPython.core.magic import register_cell_magic

ipython = get_ipython()

@register_cell_magic
def pybash(line, cell):
    ipython.run_cell_magic('bash', '', cell.format(**globals()))

In [2]:
import ezid_client_tools as ect
from ezid_client_tools.utils import ANVL

import argparse
import settings

import os
import types


In [3]:
EZID_USER = os.environ.get('EZID_USER')
EZID_PASSWD = os.environ.get('EZID_PASSWD')

if (EZID_USER is None) or (EZID_PASSWD is None):
    import settings
    EZID_USER = settings.EZID_USER
    EZID_PASSWD = settings.EZID_PASSWD

    

In [4]:
cclient = ect.ConsoleClient()

cclient.args.credentials = f"{EZID_USER}:{EZID_PASSWD}"
cclient.args.server = "p"

cclient.args.operation = ['view', 'ark:/28722/k2154wc6r']

cclient.operation()

success: ark:/28722/k2154wc6r
erc.who: Anthony Tuck
erc.what: Photo 19660118 from Italy/Poggio Civitate/Tesoro/Tesoro 2F/1966, ID:476/PC 19660118
erc.when: 2017-10-04
_owner: opencontext
_ownergroup: ucblibrary
_created: 1624495319
_updated: 1624495319
_profile: erc
_target: http://opencontext.org/media/23b1e460-9c04-4672-bf5b-91bae2b838b0
_status: public
_export: yes


In [5]:
anvl = ANVL()
client = ect.Client()

client.args.credentials = f"{EZID_USER}:{EZID_PASSWD}"
client.args.server = "p"

client.args.operation = ['view', 'ark:/28722/k2154wc6r']

In [6]:
r = client.operation()
anvl.parse_anvl_str(r.encode('utf-8'))

LastUpdatedOrderedDict([('success', 'ark'),
                        ('erc.who', 'Anthony Tuck'),
                        ('erc.what',
                         'Photo 19660118 from Italy/Poggio Civitate/Tesoro/Tesoro 2F/1966, ID'),
                        ('erc.when', '2017-10-04'),
                        ('_owner', 'opencontext'),
                        ('_ownergroup', 'ucblibrary'),
                        ('_created', '1624495319'),
                        ('_updated', '1624495319'),
                        ('_profile', 'erc'),
                        ('_target', 'http'),
                        ('_status', 'public'),
                        ('_export', 'yes')])

In [7]:
%%pybash

ezid p "{EZID_USER}:{EZID_PASSWD}" view ark:/28722/k2154wc6r 

success: ark:/28722/k2154wc6r
erc.who: Anthony Tuck
erc.what: Photo 19660118 from Italy/Poggio Civitate/Tesoro/Tesoro 2F/1966, ID:476/PC 19660118
erc.when: 2017-10-04
_owner: opencontext
_ownergroup: ucblibrary
_created: 1624495319
_updated: 1624495319
_profile: erc
_target: http://opencontext.org/media/23b1e460-9c04-4672-bf5b-91bae2b838b0
_status: public
_export: yes


In [8]:
%%pybash

curl -u "{EZID_USER}:{EZID_PASSWD}" -d format=anvl -d type=ark \
  -d permanence=real \
  -d createdAfter=2023-01-01T00:00:00Z \
  https://ezid.cdlib.org/download_request

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   134  100    64  100    70    396    433 --:--:-- --:--:-- --:--:--   858


success: https://ezid.cdlib.org/download/HrpkaWp2hzFSi4qm.txt.gz

In [9]:
%%pybash

curl -u "{EZID_USER}:{EZID_PASSWD}" \
  -d format=csv \
  -d column=_id \
  -d column=erc.who \
  -d type=ark \
  -d permanence=real \
  -d createdAfter=2023-01-01T00:00:00Z \
  https://ezid.cdlib.org/download_request


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   159  100    64  100    95    388    576 --:--:-- --:--:-- --:--:--   993


success: https://ezid.cdlib.org/download/NyftZ6WQZ2JEYFf7.csv.gz

In [ ]:
%%pybash

curl -u "{EZID_USER}:{EZID_PASSWD}" \
  -d format=csv \
  -d column=_id \
  -d column=erc.who \
  -d column=erc.what \
  -d column=erc.when \
  -d column=_owner \
  -d column=_ownergroup \
  -d column=_created \
  -d column=_updated \
  -d column=_profile \
  -d column=_target \
  -d column=_status \
  -d column=_export \
  -d type=ark \
  -d permanence=real \
  -d createdAfter=2023-04-01T00:00:00Z \
  https://ezid.cdlib.org/download_request



In [10]:
import requests
url = 'https://ezid.cdlib.org/download_request'
data = {
    'format': 'csv',
    'column': ['_id', 'erc.who', 'erc.what', 'erc.when', '_owner', '_ownergroup', '_created', '_updated', '_profile', '_target', '_status', '_export'],
    'type': 'ark',
    'permanence': 'real',
    'createdAfter': '2023-04-01T00:00:00Z'
}

response = requests.post(url, auth=(EZID_USER, EZID_PASSWD), data=data)

if response.status_code == 200:
    print(response.text)
else:
    print(f"Error: {response.status_code}")

success: https://ezid.cdlib.org/download/LBvqWJmLNWJAzB4p.csv.gz


In [11]:
response.status_code, response.text

(200, 'success: https://ezid.cdlib.org/download/LBvqWJmLNWJAzB4p.csv.gz')

# Test arks

In [12]:
TEST_NAAN = "99999"
TEST_SHOULDER = "fk4"


In [13]:
# write a public fake ARK

In [14]:
%%pybash

ezid p "{EZID_USER}:{EZID_PASSWD}" create! ark:/{TEST_NAAN}/{TEST_SHOULDER}/00/a5

success: ark:/99999/fk4/00/a5


In [15]:
%%pybash

ezid s "{EZID_USER}:{EZID_PASSWD}" create! ark:/{TEST_NAAN}/{TEST_SHOULDER}/00/a5

success: ark:/99999/fk4/00/a5


In [ ]:
client.args.server = "p"

client.args.operation = ['create!', f'ark:/{TEST_NAAN}/{TEST_SHOULDER}/00/a5']
r = client.operation()
r

In [ ]:
%%pybash

ezid p "{EZID_USER}:{EZID_PASSWD}" view ark:/{TEST_NAAN}/{TEST_SHOULDER}/00/a5

# Preregistration Shoulder

In [ ]:
# New shoulder

OC_NAAN = "28722"
OC_PREREG_SHOULDER = "r2"

def create_ark(self, id_, update=True ):
    print(f"Creating ARK for {id_}...")

# Dynamically add the create_ark method to the client object
client.create_ark = types.MethodType(create_ark, client)

# Call the create_ark method on the client object
client.create_ark("ark://")


# Analyzing OpenContext arks

In [ ]:
import pandas as pd
from pandas import DataFrame, Series, Index
import numpy as np

In [ ]:
df = pd.read_csv("data/5lrRHNLMtGqBVN8d.csv")
df.head(2)

In [ ]:
len(df)

In [ ]:
df['_status'].value_counts()